In [2]:
import os
import cv2
import math
import torch
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings

/home/jianming/conda/envs/pica37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
overall_img_path_list = []
path_prefix = "/home/jianming/work/multiface/dataset/m--20180227--0000--6795937--GHS/unwrapped_uv_1024/"
all_dir = os.listdir(path_prefix)
# print(all_dir)
for sgl_dir in all_dir:
    path_average = os.path.join(path_prefix + sgl_dir, "average")
    for image in os.listdir(path_average):
        # print(os.path.join(path_average, image))
        overall_img_path_list.append(os.path.join(path_average, image))

def calculate_l2_norm(image_path1, image_path2):
    # Load images
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)

    # Check if images are loaded
    if image1 is None or image2 is None:
        raise ValueError("One or both images could not be loaded. Check the image paths.")

    # Ensure the images are of the same size
    if image1.shape != image2.shape:
        raise ValueError("Images must have the same dimensions for L2 norm calculation.")

    # Flatten the images into vectors
    vector1 = image1.flatten()
    vector2 = image2.flatten()

    # Calculate the L2 norm (Euclidean distance)
    l2_norm = np.linalg.norm(vector1 - vector2)

    return l2_norm

In [4]:
num_images = len(overall_img_path_list)

l2_norm_expression_list = np.zeros((len(overall_img_path_list),len(overall_img_path_list)))
for i in range(num_images):
    image1 = cv2.imread(overall_img_path_list[i], cv2.IMREAD_GRAYSCALE).flatten()
    for j in range(num_images):
        image2 = cv2.imread(overall_img_path_list[j], cv2.IMREAD_GRAYSCALE).flatten()
        l2_norm_expression_list[i][j] = np.linalg.norm(image1 - image2)

In [ ]:
with open('l2_norm_unwrapp_texture_array.npy', 'wb') as f:
    np.save(f, l2_norm_expression_list)

l2_norm_threshold_list = [100, 1000, 10000, 100000]
for l2_norm_threshold in l2_norm_threshold_list:
    counts = np.max(np.sum(l2_norm_expression_list < l2_norm_threshold, axis=1))
    print(f"maximal_number_expression_within_l2norm: {counts}")